In [5]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import Queue
from threading import Timer
from threading import Thread

In [ ]:
#https://pymotw.com/2/threading/

In [7]:
#https://stackoverflow.com/questions/12435211/python-threading-timer-repeat-function-every-n-seconds
#Repeated Timer
class MyThread(Thread):
    def __init__(self, event):
        Thread.__init__(self)
        self.stopped = event

    def run(self):
        while not self.stopped.wait(0.5):
            print("my thread")
            
            #Call a function
            fastThread()

In [ ]:
def getAudio():
    myRecording1 = sd.rec(duration * fs, samplerate=fs, channels=1)
    sd.wait()
    
    #Each sample is of type list. Convert from list to int
    bClip = []
    for i in range(len(myRecording1)):
        bClip.append( myRecording1[i][0] )
    
    return bClip

In [ ]:
def fastThread():
    #Get clip
    aClip = getAudio()
    
    #Push to queue
    processQueue.put( aClip )
    
    #Maybe delay a split second - Done in Thread Class
    
    #Call thread again

In [ ]:
def getFFT(_aClip):
    
    #Get data
    #data = aClips[i]
    data = _aClip

    #Smooth the FFT by windowing data
    data = data * np.hanning(len(data))

    #Preform FFT and take only real data
    fft = abs(np.fft.fft(data).real)

    #Keep only first half
    fft = fft[:int(len(fft))/2]

    #Create Frequency list
    freq = np.fft.fftfreq( (fs * duration) ,1.0/fs)

    #Keep only first half
    freq = freq[:int(len(freq))/2] 

    #At this point, the length of the fft/freq is cut in reduced from 2048 to 1024. But the cut off frequency is actually 512.
    #The freq axis shows why. Values are now 0, 0.5, 1, 1.5Hz...
    #Therefore drop every other value
    fft = fft[0::2]
    freq = freq[0::2]

    #Save for later
    #ffts.append( fft )
    
    return fft

In [ ]:
def predictFFT:
    #train(_nips, _nhid 14, _nops, _rate 0.5, _anneal 0.99, _data, _iter 32
    train(512, 14, 2, 0.5, 0.99, 'woofsAndMeows.data', 64)


In [ ]:
#def formatDataForPrediction


In [ ]:
#Train

In [ ]:
#Create queues
processQueue = Queue.Queue()

#Create events
stopFlag = Event()

#Create threads
thread = MyThread(stopFlag)
thread.start()

# this will stop the timer
#stopFlag.set()

In [ ]:
#While loop
while True:
    
    #Wait for something in Queue
    if not processQueue.empty(): 
        aClip = processQueue.get( aClip )

    #Get FFT
    clipFFT = getFFT( aClip )

    #formatDataForPrediction
    
    #Predict
    var0, var1 = predictFFT( clipFFT )

    #Found woof
    if var0 > 0.7 and var1 < 0.4:
        print 'Found Woof'
    #Found meow   
    elif var1 > 0.4 and var0 > 0.7:
        print 'Meow'
        